In [18]:
print("Hola mundo")

Hola mundo


In [30]:
# 🤓 Iniciamos con la configuración y extracción de las librerías senseval, punk stopwords wordnet,
import os
import nltk
from nltk.corpus import senseval
import re
nltk.download('senseval') # Para el corpus Senseval 2
nltk.download('punkt')    # Para tokenización
nltk.download('stopwords') # Para palabras vacías (stop words)
nltk.download('wordnet')  # Aunque no se use directamente para los sentidos, es útil para entender WordNet.
nltk.download('averaged_perceptron_tagger') # Para etiquetado POS si se necesita, aunque Senseval ya viene con ello.

[nltk_data] Downloading package senseval to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package senseval is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Aqui Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
Listamos todas las instancias
senseval.fileids()

['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']

In [21]:
hard_instances = senseval.instances('hard.pos')
hard_instances = [instance for instance in hard_instances if instance.senses]  # Filtrar instancias con sentidos
print(f"Total de instancias con sentidos: {len(hard_instances)}")

Total de instancias con sentidos: 4333


In [36]:
file_path = 'corpus_etiquetado.txt'

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        corpus_content = f.read()
    print(f"Archivo '{file_path}' cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no se encontró. Asegúrate de que está en la misma carpeta que tu script.")
except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")

Archivo 'corpus_etiquetado.txt' cargado exitosamente.


In [40]:
import re

def parse_corpus_content(content):
    """
    Procesa el contenido del corpus para extraer pares (palabra, etiqueta).
    Maneja etiquetas de documento, líneas vacías y el formato específico.
    """
    corpus = []
    
    for linea in content.splitlines():
        linea = linea.strip()
        
        # Saltamos etiquetas de documento y líneas vacías.
        if linea.startswith("<doc") or linea.startswith("</doc>") or linea == "":
            continue
        
        # Saltar 'Fp' si aparece en una línea separada (inconsistencia del corpus).
        if linea == "Fp":
            continue
        
        datos = linea.split("\t")
        
        # Formato esperado: palabra \t lema \t etiqueta
        if len(datos) == 3:
            word = datos[0]
            tag = datos[2]
            corpus.append((word, tag))
        # Caso específico: . \t . \t Fp
        elif len(datos) == 2 and datos[0] == '.' and datos[1] == 'Fp':
            word = datos[0]
            tag = datos[1] 
            corpus.append((word, tag))
        # Las líneas que no encajan se omiten.
        else:
            pass 

    return corpus


In [42]:
parse_corpus_content(corpus_content) # procesamos el contenido del corpus de manera organizada
# Procesamos el contenido del corpus para extraer pares (palabra, etiqueta)

[('Habla', 'VMIP3S0'),
 ('con', 'SP'),
 ('el', 'DA'),
 ('enfermo', 'NCMS000'),
 ('grave', 'AQ0CS00'),
 ('de', 'SP'),
 ('trasplantes', 'NCMN000'),
 ('.', 'Fp'),
 ('El', 'DA'),
 ('enfermo', 'NCMS000'),
 ('grave', 'AQ0CS00'),
 ('habla', 'VMIP3S0'),
 ('de', 'SP'),
 ('trasplantes', 'NCMN000'),
 ('.', 'Fp'),
 ('La', 'DA'),
 ('película', 'NCFS000'),
 ('fue', 'VAIP3S0'),
 ('nominada', 'VMP00SF'),
 ('al', 'SP+DA'),
 ('Oscar', 'NP00000'),
 ('.', 'Fp'),
 ('Luis', 'NP00000'),
 ('Buñuel', 'NP00000'),
 ('es', 'VSIP3S0'),
 ('director', 'NCMS000'),
 ('español', 'AQ0MS00'),
 ('.', 'Fp'),
 ('El', 'DA'),
 ('niño', 'NCMS000'),
 ('corre', 'VMIP3S0'),
 ('en', 'SP'),
 ('el', 'DA'),
 ('parque', 'NCMS000'),
 ('.', 'Fp'),
 ('Una', 'DI'),
 ('flor', 'NCFS000'),
 ('roja', 'AQ0FS00'),
 ('decoraba', 'VIIIS0S'),
 ('la', 'DA'),
 ('mesa', 'NCFS000'),
 ('.', 'Fp'),
 ('Ella', 'PP3FS000'),
 ('canta', 'VMIP3S0'),
 ('una', 'DI'),
 ('canción', 'NCFS000'),
 ('alegre', 'AQ0CS00'),
 ('.', 'Fp'),
 ('Ellos', 'PP3MP000'),
 ('juega

In [45]:
corpus = parse_corpus_content(corpus_content)
print("Corpus cargado. Número de tokens:", len(corpus))
print("Primeros 5 registros:", corpus[:5])

Corpus cargado. Número de tokens: 137
Primeros 5 registros: [('Habla', 'VMIP3S0'), ('con', 'SP'), ('el', 'DA'), ('enfermo', 'NCMS000'), ('grave', 'AQ0CS00')]


In [43]:
# --- Analysis of "hard" and "serve" (using Senseval for these specific words) ---

# Load Senseval instances for the word "hard"
hard_instances = []
try:
    # Load Senseval instances for the word "hard"
    print("Cargando instancias de 'hard' desde Senseval...")
    hard_instances = senseval.instances('hard.pos')
    # Filter instances to ensure they have senses, as some might not
    hard_instances = [instance for instance in hard_instances if instance.senses]
    print(f"Total Senseval 'hard' instances with senses: {len(hard_instances)}")
except Exception as e:
    print(f"Could not load 'hard.pos' from senseval, skipping 'hard' analysis for senses: {e}")


Cargando instancias de 'hard' desde Senseval...
Total Senseval 'hard' instances with senses: 4333


In [44]:
# Inicializamos las instancias de "serve" de Senseval
serve_instances = []
try:
    # Load instances for "serve" from Senseval
    serve_instances = senseval.instances('serve.pos')
    # Filter instances to ensure they have senses, as some might not
    serve_instances = [instance for instance in serve_instances if instance.senses]
    print(f"Total Senseval 'serve' instances with senses: {len(serve_instances)}")
except Exception as e:
    print(f"Could not load 'serve.pos' from senseval, skipping 'serve' analysis for senses: {e}")



Total Senseval 'serve' instances with senses: 4378


In [ ]:
print("\n--- 1. Posibles sentidos y etiquetas para 'hard and 'serve' ---")

hard_senses = set()
for instance in hard_instances:
    for sense in instance.senses:
        hard_senses.add(sense)
print(f"\n'hard' has {len(hard_senses)} possible senses: {hard_senses}")

serve_senses = set()
for instance in serve_instances:
    for sense in instance.senses:
        serve_senses.add(sense)
print(f"\n'serve' has {len(serve_senses)} possible senses: {serve_senses}")




--- 1. Posibles sentidos y etiquetas para 'hard and 'serve' ---

'hard' has 3 possible senses: {'HARD1', 'HARD3', 'HARD2'}

'serve' has 4 possible senses: {'SERVE2', 'SERVE10', 'SERVE12', 'SERVE6'}


In [26]:
# 2. Number of instances for each sense
print("\n--- 2. Number of instances for each sense ---")

hard_sense_counts = {}
for instance in hard_instances:
    for sense in instance.senses:
        hard_sense_counts[sense] = hard_sense_counts.get(sense, 0) + 1
print("\n'hard' sense counts:")
for sense, count in hard_sense_counts.items():
    print(f"  Sense '{sense}': {count} instances")

serve_sense_counts = {}
for instance in serve_instances:
    for sense in instance.senses:
        serve_sense_counts[sense] = serve_sense_counts.get(sense, 0) + 1
print("\n'serve' sense counts:")
for sense, count in serve_sense_counts.items():
    print(f"  Sense '{sense}': {count} instances")




--- 2. Number of instances for each sense ---

'hard' sense counts:
  Sense 'HARD1': 3455 instances
  Sense 'HARD2': 502 instances
  Sense 'HARD3': 376 instances

'serve' sense counts:
  Sense 'SERVE10': 1814 instances
  Sense 'SERVE12': 1272 instances
  Sense 'SERVE2': 853 instances
  Sense 'SERVE6': 439 instances


In [34]:
# 3. Grammatical forms for "hard" and "serve"
print("\n--- 3. Grammatical forms for 'hard' and 'serve' ---")

# For 'hard'
hard_forms = set()
for instance in hard_instances:
    # instance.context might contain (word, tag) tuples or just words.
    # We'll try to handle both cases by checking the type of 'item'
    for item in instance.context:
        word_to_check = ''
        if isinstance(item, tuple):
            word_to_check = item[0]  # Get the word from the tuple
        elif isinstance(item, str):
            word_to_check = item   # 'item' is already the word string
        
        if word_to_check: # Ensure it's not empty
            if word_to_check.lower() == 'hard' or word_to_check.lower() == 'harder' or word_to_check.lower() == 'hardest':
                hard_forms.add(word_to_check.lower())
print(f"\nFormatos gramaticales en 'hard' encontrados: {hard_forms}")

# For 'serve'
serve_forms = set()
for instance in serve_instances:
    for item in instance.context:
        word_to_check = ''
        if isinstance(item, tuple):
            word_to_check = item[0]
        elif isinstance(item, str):
            word_to_check = item
            
        if word_to_check:
            if word_to_check.lower().startswith('serve'): # Catch serve, serves, served, serving, etc.
                serve_forms.add(word_to_check.lower())
print(f"\nFormatos gramaticales en 'server' encontrados: {serve_forms}")


--- 3. Grammatical forms for 'hard' and 'serve' ---

Formatos gramaticales en 'hard' encontrados: {'hard', 'harder', 'hardest'}

Formatos gramaticales en 'server' encontrados: {'serve', 'server', 'served', 'serves'}


In [ ]:
print(f"\nGrammatical forms of 'serve' found: {serve_forms}")
# --- Analysis of the uploaded corpus_etiquetado.txt ---
print("\n--- Analysis of corpus_etiquetado.txt format ---")

# Regex to parse each line and validate format
# This regex is specifically designed for the format "word\tlemma\ttag"
line_pattern = re.compile(r'^(.*?)\t(.*?)\t([A-Z]{2,}[A-Z0-9]{0,})$')

# Parse the provided corpus and check for inconsistencies
inconsistent_lines = []
parsed_docs = {}
current_doc_id = None

# lectura del contenido del archivo corpus_etiquetado.txt 
for line in corpus_content.splitlines():
    line = line.strip()
    if line.startswith('<doc id='):
        match = re.search(r'<doc id="(\d+)">', line)
        if match:
            current_doc_id = match.group(1)
            parsed_docs[current_doc_id] = []
        else:
            inconsistent_lines.append(f"Invalid doc ID format: {line}")
    elif line == '</doc>':
        current_doc_id = None
    elif line and not line.startswith('<'): # Process actual content lines
        parts = line.split('\t')
        if len(parts) == 3:
            word, lemma, tag = parts
            # Basic tag validation: should start with 2 uppercase letters (e.g., VM, NC)
            if not re.match(r'^[A-Z]{2,}', tag):
                inconsistent_lines.append(f"Invalid tag format: {line} (Doc ID: {current_doc_id})")
            if current_doc_id:
                parsed_docs[current_doc_id].append((word, lemma, tag))
        elif len(parts) == 2 and parts[0] == '.' and parts[1] == 'Fp': # Special case for "." "Fp"
            if current_doc_id:
                # Store it in a way that reflects it's a special tag
                parsed_docs[current_doc_id].append((parts[0], parts[1], 'Fp'))
        else:
            inconsistent_lines.append(f"Incorrect number of tabs or format: {line} (Doc ID: {current_doc_id})")

# Check if all instances in the provided corpus follow the format
print("\n--- 4. Checking corpus_etiquetado.txt format consistency ---")

if not inconsistent_lines:
    print("Todas las inconsistencias en el modulo 'corpus_etiquetado.txt' que aparecen el  formato (word\\tlemma\\ttag).")
else:
    print("Por si se encuentra alguna inconsistencia 'corpus_etiquetado.txt':")
    for i, problem in enumerate(inconsistent_lines[:10]): # Show first 10 examples
        print(f"  {i+1}. {problem}")
    if len(inconsistent_lines) > 10:
        print(f"  ...and {len(inconsistent_lines) - 10} more inconsistencies.")

# Example of a well-formatted line from your corpus for reference
print("\nEjemplo como lo pide la guia con (word\\tlemma\\ttag):")
print("Habla\\thablar\\tVMIP3S0")
print("con\\tcon\\tSP")
print("el\\tel\\tDA")


Grammatical forms of 'serve' found: {'serve', 'server', 'served', 'serves'}

--- Analysis of corpus_etiquetado.txt format ---

--- 4. Checking corpus_etiquetado.txt format consistency ---
Some inconsistencies found in 'corpus_etiquetado.txt':
  1. Invalid tag format: .	.	Fp (Doc ID: 1)
  2. Invalid tag format: .	.	Fp (Doc ID: 2)
  3. Invalid tag format: .	.	Fp (Doc ID: 3)
  4. Invalid tag format: .	.	Fp (Doc ID: 4)
  5. Invalid tag format: .	.	Fp (Doc ID: 5)
  6. Invalid tag format: .	.	Fp (Doc ID: 6)
  7. Invalid tag format: .	.	Fp (Doc ID: 7)
  8. Invalid tag format: .	.	Fp (Doc ID: 8)
  9. Invalid tag format: .	.	Fp (Doc ID: 9)
  10. Invalid tag format: .	.	Fp (Doc ID: 10)
  ...and 10 more inconsistencies.

Example of expected format (word\tlemma\ttag):
Habla\thablar\tVMIP3S0
con\tcon\tSP
el\tel\tDA


In [46]:
def calcular_probabilidades_emision(corpus):
    emision = {}
    total_etiquetas = {}
    for token, etiqueta in corpus:
        if etiqueta not in emision:
            emision[etiqueta] = {}
            total_etiquetas[etiqueta] = 0
        emision[etiqueta][token] = emision[etiqueta].get(token, 0) + 1
        total_etiquetas[etiqueta] += 1
    for etiqueta in emision:
        for token in emision[etiqueta]:
            emision[etiqueta][token] /= total_etiquetas[etiqueta]
    return emision

def calcular_probabilidades_transicion(corpus):
    transicion = {}
    total_transiciones = {}
    anterior = None
    for _, etiqueta in corpus:
        if anterior is not None:
            if anterior not in transicion:
                transicion[anterior] = {}
                total_transiciones[anterior] = 0
            transicion[anterior][etiqueta] = transicion[anterior].get(etiqueta, 0) + 1
            total_transiciones[anterior] += 1
        anterior = etiqueta
    for etiqueta in transicion:
        for siguiente in transicion[etiqueta]:
            transicion[etiqueta][siguiente] /= total_transiciones[etiqueta]
    return transicion

def calcular_probabilidades_iniciales(corpus):
    iniciales = {}
    total = 0
    if corpus:
        iniciales[corpus[0][1]] = 1
        total = 1
    for etiqueta in iniciales:
        iniciales[etiqueta] /= total
    return iniciales

emision = calcular_probabilidades_emision(corpus)
transicion = calcular_probabilidades_transicion(corpus)
estado_inicial = calcular_probabilidades_iniciales(corpus)

In [47]:
def viterbi(frase, estados, transicion, emision, estado_inicial):
    V = [{}]
    path = {}

    for estado in estados:
        V[0][estado] = estado_inicial.get(estado, 1e-6) * emision.get(estado, {}).get(frase[0], 1e-6)
        path[estado] = [estado]

    for t in range(1, len(frase)):
        V.append({})
        new_path = {}
        for y in estados:
            (prob, estado_max) = max(
                [(V[t-1][y0] * transicion.get(y0, {}).get(y, 1e-6) * emision.get(y, {}).get(frase[t], 1e-6), y0)
                 for y0 in estados], key=lambda x: x[0])
            V[t][y] = prob
            new_path[y] = path[estado_max] + [y]
        path = new_path

    n = len(frase) - 1
    (prob, estado_max) = max([(V[n][y], y) for y in estados], key=lambda x: x[0])
    return (prob, path[estado_max])

In [48]:
frase_prueba = ['Habla', 'con', 'el', 'enfermo', 'grave', 'de', 'trasplantes', '.']
estados = list(set([etiqueta for _, etiqueta in corpus]))

probabilidad, ruta = viterbi(frase_prueba, estados, transicion, emision, estado_inicial)
print("Frase:", frase_prueba)
print("Ruta más probable:", ruta)
print("Probabilidad total:", probabilidad)

Frase: ['Habla', 'con', 'el', 'enfermo', 'grave', 'de', 'trasplantes', '.']
Ruta más probable: ['VMIP3S0', 'SP', 'DA', 'NCMS000', 'AQ0CS00', 'SP', 'NCMN000', 'Fp']
Probabilidad total: 1.8533917043090838e-08


In [54]:
import pandas as pd

# (Assuming 'emision', 'transicion', 'frase_prueba', and 'ruta' are defined as above or loaded elsewhere)

# Create DataFrame for Emisión (Emission Probabilities)
# Rows will be words, columns will be tags
df_emision = pd.DataFrame.from_dict(emision, orient='index').fillna(0)
print("DataFrame de Emisión:")
print(df_emision)
print("\n" + "="*50 + "\n")


DataFrame de Emisión:
             Habla     habla     corre     canta    brilla    avanza  \
VMIP3S0   0.142857  0.142857  0.142857  0.142857  0.142857  0.142857   
SP        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
DA        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCMS000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
AQ0CS00   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCMN000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Fp        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCFS000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VAIP3S0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VMP00SF   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
SP+DA     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NP00000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VSIP3S0   0.000000  0.000000  0.000000  0.

In [ ]:
# Create DataFrame for Transición (Transition Probabilities)
# Rows will be current tags, columns will be next tags
df_transicion = pd.DataFrame.from_dict(transicion, orient='index').fillna(0)
print("DataFrame de Transición:")
print(df_transicion)
print("\n" + "="*50 + "\n")


DataFrame de Transición:
                SP        DI        RG   NCMP000        DA   NCMN000  \
VMIP3S0   0.428571  0.142857  0.285714  0.142857  0.000000  0.000000   
NCMS000   0.105263  0.052632  0.000000  0.000000  0.000000  0.000000   
AQ0CS00   0.142857  0.000000  0.000000  0.000000  0.000000  0.000000   
VMIP3P0   0.333333  0.000000  0.000000  0.000000  0.000000  0.000000   
RG        0.750000  0.000000  0.000000  0.000000  0.000000  0.000000   
VIIIS3S   0.500000  0.500000  0.000000  0.000000  0.000000  0.000000   
Fp        0.000000  0.157895  0.052632  0.000000  0.315789  0.000000   
VSIP3S0   0.000000  0.333333  0.000000  0.000000  0.000000  0.000000   
VMIS3S0   0.000000  0.000000  0.500000  0.000000  0.500000  0.000000   
AQ0MP00   0.000000  0.000000  0.000000  0.500000  0.000000  0.000000   
SP        0.000000  0.000000  0.000000  0.000000  0.636364  0.181818   
VIIIS0S   0.000000  0.000000  0.000000  0.000000  1.000000  0.000000   
DA        0.000000  0.000000  0.000000 

In [56]:

# Create DataFrame for Viterbi result
# Shows the input phrase words and their predicted tags
df_viterbi = pd.DataFrame({
    "Palabra": frase_prueba,
    "Etiqueta": ruta
})
print("DataFrame Viterbi (Palabra y Etiqueta):")
print(df_viterbi)


DataFrame Viterbi (Palabra y Etiqueta):
       Palabra Etiqueta
0        Habla  VMIP3S0
1          con       SP
2           el       DA
3      enfermo  NCMS000
4        grave  AQ0CS00
5           de       SP
6  trasplantes  NCMN000
7            .       Fp


In [ ]:
df_emision.to_excel("tabla_emision.xlsx", engine='openpyxl')
df_transicion.to_excel("tabla_transicion.xlsx", engine='openpyxl')
df_viterbi.to_excel("ruta_viterbi.xlsx", engine='openpyxl')

from IPython.display import FileLink, display
display(FileLink("tabla_emision.xlsx"))
display(FileLink("tabla_transicion.xlsx"))
display(FileLink("ruta_viterbi.xlsx"))